In [1]:
import pystac

items = pystac.ItemCollection.from_file("results.json")

assets = items[-1].assets
visual_href = assets["visual"].href
scl_href = assets["SCL"].href

In [2]:
import rioxarray

scl = rioxarray.open_rasterio(scl_href)
visual = rioxarray.open_rasterio(visual_href)
print(scl.rio.resolution(), visual.rio.resolution())

(20.0, -20.0) (10.0, -10.0)


In [3]:
visual = rioxarray.open_rasterio(visual_href, overview_level=0)
print(visual.rio.resolution())

(20.0, -20.0)


In [ ]:
%%time
scl = scl.load()
visual = visual.load()

In [ ]:
visual.plot.imshow(figsize=(10, 10))
scl.squeeze().plot.imshow(levels=range(13), figsize=(12, 10))

In [ ]:
%%time
mask = scl.squeeze().isin([8, 9])
visual_masked = visual.where(~mask, other=visual.rio.nodata)
visual_masked.rio.to_raster("band_masked.tif")

In [ ]:
visual_masked.plot.imshow(figsize=(10, 10))

In [ ]:
blue_band_href = assets["B02"].href
blue_band = rioxarray.open_rasterio(blue_hand_href, chunks=(1, 4000, 4000))

In [ ]:
# Exercise: Chunk sizes matter
import rasterio
with rasterio.open(visual_href) as r:
    if r.is_tiled:
        print(f"Chunk size: {r.block_shapes}")

In [ ]:
scl = rioxarray.open_rasterio(scl_href, lock=False, chunks=(1, 2048, 2048))
visual = rioxarray.open_rasterio(visual_href, overview_level=0, lock=False, chunks=(3, 2048, 2048))

In [ ]:
%%time
scl = scl.persist(scheduler="threads", num_workers=4)
visual = visual.persist(scheduler="threads", num_workers=4)

In [ ]:
from threading import Lock

In [ ]:
%%time
mask = scl.squeeze().isin([8, 9])
visual_masked = visual.where(~mask, other=0)
visual_store = visual_masked.rio.to_raster("band_masked.tif", tiled=True, lock=Lock(), compute=False)

In [ ]:
import dask
dask.visualize(visual_store)

In [ ]:
%%time
visual_store.compute(scheduler="threads", num_workers=4)